In [46]:
import re
import json
import os
import csv
import sys

from nltk.tokenize import wordpunct_tokenize
import numpy as np
import random
from gensim.models.wrappers.fasttext import FastText as FT_wrapper

import pickle

csv.field_size_limit(sys.maxsize)

9223372036854775807

In [20]:
negatives = []

In [26]:
def preprocess_forum(f):
    lines = []
    for line in f:        
        t = '\n'.join(' '.join(wordpunct_tokenize(l)) for l in json.loads(line)['text'].strip().splitlines() if l.strip())
        if t:
            lines.append(t)
    return lines
    

In [27]:
with open('forum_scraper/forummoskva/forummoskva.jsonl') as f:
    texts = preprocess_forum(f)    
    negatives.extend(texts)
    print(len(texts))

1238816


In [28]:
with open('forum_scraper/forummoskva/dromru.jsonl') as f:
    texts = preprocess_forum(f)    
    negatives.extend(texts)
    print(len(texts))    

15211420


In [29]:
with open('forum_scraper/forummoskva/vashdomru.jsonl') as f:
    texts = preprocess_forum(f)    
    negatives.extend(texts)  
    print(len(texts))  # 300k

65593


In [30]:
with open('forum_scraper/forummoskva/antiwomenru.jsonl') as f:
    texts = preprocess_forum(f)    
    negatives.extend(texts)  
    print(len(texts))  # 1.8m

111866


In [31]:
with open('forum_scraper/forummoskva/womenru.jsonl') as f:
    texts = preprocess_forum(f)    
    negatives.extend(texts)  
    print(len(texts))

13025440


In [39]:
with open('dataset.tsv', 'w') as f:
    csvw = csv.writer(f, delimiter='\t')    
    for line in negatives:
        csvw.writerow([0, line])   

del negatives

In [2]:
DATA = '/home/marat/data/taskrecog'

positives = []
with open(os.path.join(DATA, 'youdo', 'youdo.txt')) as f:
    csvr = csv.reader(f, delimiter=',')
    for row in csvr:
        t = '\n'.join(' '.join(wordpunct_tokenize(l)) for l in row[1].strip().splitlines() if l)
        positives.append(t)

KeyboardInterrupt: 

In [3]:
with open('dataset.tsv', 'a') as f:
    csvw = csv.writer(f, delimiter='\t')  
    for line in positives:
        csvw.writerow([1, line])      
        
del positives

In [2]:
data = []
with open('dataset.tsv') as f:
    csvr = csv.reader(f, delimiter='\t')
    for label, text in csvr:
        data.append((label, text))
        
random.shuffle(data)

In [5]:
train_n = int(len(data) * 0.8)

with open('train.tsv', 'w') as f:
    csvw = csv.writer(f, delimiter='\t')
    for label, text in data[:train_n]:
        csvw.writerow([label, text])

with open('test.tsv', 'w') as f:
    csvw = csv.writer(f, delimiter='\t')
    for label, text in data[train_n:]:
        csvw.writerow([label, text])



In [2]:
train_X = []
train_y = []
with open('train.tsv') as f:
    csvr = csv.reader(f, delimiter='\t')
    for label, text in csvr:
        train_X.append(text)
        train_y.append(int(label))
        
train_y = np.array(train_y)

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

tfidf = TfidfVectorizer()
train_X = tfidf.fit_transform(train_X)

train_X.shape

/home/marat/.local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


(26187670, 5406931)

In [43]:
%%time

m = LinearSVC(C=0.1)
m.fit(train_X, train_y)

CPU times: user 1min, sys: 372 ms, total: 1min
Wall time: 1min


In [39]:
(m.predict(train_X) == train_y).mean()

0.9986286294275131

In [8]:
pos = train_y == 1
(m.predict(train_X[pos]) == train_y[pos]).mean()

0.6425858402732451

In [9]:
(m.predict(train_X[~pos]) == train_y[~pos]).mean()


0.9997326597886819

In [19]:
m.predict(tfidf.transform(['нужно']))

/home/marat/.local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


array([1])

In [12]:
tfidf.transform(['нужно'])

/home/marat/.local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


<1x5406931 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [21]:
test_X = []
test_y = []
with open('test.tsv') as f:
    csvr = csv.reader(f, delimiter='\t')
    for label, text in csvr:
        test_X.append(text)
        test_y.append(int(label))
        
test_y = np.array(test_y)
test_X = tfidf.transform(test_X)

/home/marat/.local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [44]:
print('Accuracy  ', (m.predict(test_X) == test_y).mean(), (m.predict(train_X) == train_y).mean(), sep='\t')
test_pos = test_y == 1
train_pos = train_y == 1
print('Pos Accuracy', (m.predict(train_X[train_pos]) == train_y[train_pos]).mean(), (m.predict(test_X[test_pos]) == test_y[test_pos]).mean(), sep='\t')
print('Neg Accuracy', (m.predict(train_X[~train_pos]) == train_y[~train_pos]).mean(), (m.predict(test_X[~test_pos]) == test_y[~test_pos]).mean(), sep='\t')

# 0.1


# 1
#   Accuracy	0.9966766041670294	0.9979463617801813
# Pos Accuracy	0.974006711709939	0.9588633251535406
# Neg Accuracy	0.9993740346497515	0.9989338185724046

# 10
#   Accuracy  	0.9963909735848226	0.9986286294275131
# Pos Accuracy	0.9831874787121008	0.9592863243807536
# Neg Accuracy	0.9995494829858005	0.9986058872573433

Accuracy  	0.9961985166149935	0.9965794971450305
Pos Accuracy	0.9533945972499087	0.9494597269485757
Neg Accuracy	0.9991548860823989	0.998988527745401


In [47]:
model_wrapper = FT_wrapper.train('/home/marat/reps/fastText-0.1.0/fasttext', corpus_file='dataset.tsv')

OSError: [Errno 12] Cannot allocate memory

In [7]:
model_wrapper

In [12]:
model_wrapper.

array([-0.42716983, -0.17344135,  0.3371355 , -0.6757883 , -0.5622692 ,
       -0.05977966, -0.12417972, -0.00342955, -0.29592362, -0.24564536,
       -0.7342647 , -0.10244312,  0.07394643, -0.58113915,  0.6122555 ,
       -0.32334843,  0.29416338, -0.2337593 ,  0.37154803, -0.18393017,
        0.21267232, -0.31727967, -0.45376766,  0.35149536, -0.182277  ,
        0.56808096,  0.34202024,  0.30292895, -0.22532487,  0.2211067 ,
        0.34384692,  0.52345836, -0.00901887,  0.0732601 ,  0.25762138,
        0.0095869 , -0.5043995 , -0.12006348, -0.14211147, -0.02660525,
       -0.91419667, -0.07949968,  0.13933851,  0.5112861 ,  0.21143593,
        0.06091765,  0.48951468,  0.33962092, -0.01808789, -0.03122145,
        0.24973479, -0.8743096 ,  0.75266916, -0.13190462, -0.01214689,
       -0.29096735,  0.3937865 ,  0.5269838 , -0.04508137, -0.09397663,
       -0.09388046,  0.6086231 , -0.21564537,  0.32193092, -0.36781594,
        0.15000036, -0.16726679,  0.3073794 , -0.00366162,  0.10

In [18]:
model_wrapper.most_similar('=(')

[('Юлус', 0.93408203125),
 ('=(,', 0.8703312277793884),
 ('=("', 0.869441032409668),
 (';(', 0.8526694178581238),
 (':(:(:("', 0.8471227288246155),
 (':(', 0.8459991216659546),
 ('дальше:(', 0.8446086049079895),
 (':("', 0.8394126296043396),
 ('((', 0.838272213935852),
 (':(:(:(', 0.8340241312980652)]